In [1]:
import numpy as np 
import seaborn as sns
import pandas as pd 
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px
import yfinance as yf
import quandl
from fredapi import Fred

pd.set_option("display.max_rows",200)
sns.set(rc={'figure.figsize':(16,10)})
fred_key = "df4910b2cad947d95cf6ab16ba11d74d"
fred = Fred(api_key = fred_key)
quandl.ApiConfig.api_key = 'Qq5R29Xiqp2yUbb9dzNq'

# geting the macroeconomics code and there transformations

In [42]:
MacroCode=pd.read_csv(r'..\Data\Macro Variables.csv')
MacroCode.replace({'Average HOUST':'HOUST','S&P 500':'SP500'},inplace=True)

In [21]:
import requests

def varialbe_name(variable_id):
    # FRED API base URL

    # FRED API key (replace with your own key)
    api_key = "df4910b2cad947d95cf6ab16ba11d74d"

    # Endpoint URL to get variable details
    url = f"https://api.stlouisfed.org/fred/series?series_id={variable_id}&api_key={api_key}&file_type=json"

    response = requests.get(url)
    response_json = response.json()
    try:
        name=response_json['seriess'][0]['title']
    except Exception:
        name='N/a'
    return name


In [43]:
MacroCode['Description']=MacroCode['Variable'].apply(varialbe_name)

MacroCode['Variable']=MacroCode['Variable'].apply(lambda x: "\\textbf{"+f"{x}"+"}")

In [51]:
MacroCode[['Variable','Description', 'Transformation']].to_clipboard()

In [3]:
errors=[]
indicators={}
freq={}
for code in MacroCode['Variable']:
    if not ('S&P' in code):
        try:
            col=fred.get_series(code).to_frame(code).squeeze()
            if code == 'CPFF':
                indicators[code]=(col) #('MS').last()
            elif code == "ICSA":
                indicators[code]=(col) #('MS').sum()
            elif code == "SP500":
                indicators[code]=(col) #('MS').first()
            else:
                indicators[code]=(col) #  #('M').interpolate()
        except ValueError:
            errors.append(code)
indicators['S&P: indust']=(yf.download('^SP500-20')['Close'].to_frame('S&P: indust').squeeze()) #("MS").last()
indicators['S&P div yield']=(quandl.get("MULTPL/SP500_DIV_YIELD_MONTH").squeeze().to_frame('S&P div yield').squeeze()) #("MS").last()
indicators['S&P PE ratio']=(quandl.get("MULTPL/SP500_PE_RATIO_MONTH").squeeze().to_frame('S&P PE ratio').squeeze()) #("MS").first()


[*********************100%***********************]  1 of 1 completed


In [4]:
if len(errors)>0:
    raise Exception("Erorrs found")
data=pd.concat(indicators,axis=1).copy().fillna(method='ffill') #.loc["1971":].dropna(how='all').fillna(method='ffill')
# data.to_csv('Data/HistoricalVariables.csv')
print(data.shape)
data.isna().sum().sort_values()/data.shape[0]

(13703, 124)


S&P PE ratio       0.000000
S&P div yield      0.000073
AMBSL              0.082318
BAA                0.084069
AAA                0.084069
INDPRO             0.084069
IPMANSICS          0.084069
TB3MS              0.110341
USCONS             0.119098
MANEMP             0.119098
DMANEMP            0.119098
NDMANEMP           0.119098
SRVPRD             0.119098
USTRADE            0.119098
USWTRADE           0.119098
USGOOD             0.119098
USFIRE             0.119098
USGOVT             0.119098
CES3000000008      0.119098
USTPU              0.119098
PAYEMS             0.119098
AWHMAN             0.119098
IPMAT              0.119098
IPCONGD            0.119098
IPFPNSS            0.119098
IPFINAL            0.119098
DTCTHFNM           0.126104
DTCOLNVHFNM        0.126104
TOTALSL            0.126104
NONREVSL           0.126104
WTISPLC            0.131358
CES0600000007      0.133110
CES2000000008      0.133110
IPDCONGD           0.133110
CES0600000008      0.133110
CPITRNSL           0

In [5]:
# data.to_csv('..\Data\HistoricalVariables.csv')


# Data transformation

In [6]:
def deltaX(data):
    return data.diff()

def delta2X(data):
    return data-2*data.shift(1)+data.shift(2)

def logdata(data):
    return np.log(data)

def difflog(data):
    return np.log(data).diff()

def difflog2(data):
    return np.log(data)-2*np.log(data.shift(1))+np.log(data.shift(2))

def diffpercent(data):
    return (data/data.shift() - 1) - (data.shift()/data.shift(2) - 1)



transformation={1:(lambda x: x),
                2:deltaX,
                3:delta2X,
                4:logdata,
                5:difflog,
                6:difflog2,
                7:diffpercent,
                }

In [7]:
df_transformed=pd.DataFrame()

In [8]:
for column in data.columns:
    type=MacroCode[MacroCode['Variable'] == column]['Transformation'].iloc[0]
    df_transformed[column]=transformation[type](data[column])
    df_transformed=df_transformed.copy()

In [ ]:
df_transformed.to_csv('../Data/Transformed HistoricalVariables.csv')
